In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  

import numpy as np
from tools import *

# Partie 1 : SIFT

In [ ]:
# example images
I = read_grayscale('data/tools.tiff')
I2 = read_grayscale('data/Scene/CALsuburb/image_0205.jpg')
plt.imshow(I)

In [ ]:
def compute_grad(I):
    h_x = 1/2*np.array([[-1], [0], [1]])
    h_y = 1/2*np.array([[1], [2], [1]])
    Ix = conv_separable(I, h_y, h_x)
    Iy = conv_separable(I, h_x, h_y)
    return Ix, Iy

# example d'affichage du résultat
Ix, Iy = compute_grad(I)
plt.imshow(Ix)
plt.colorbar()
plt.show()
plt.imshow(Iy)
plt.colorbar()
plt.show()

In [ ]:
def compute_grad_mod_ori(I):
    Ix, Iy = compute_grad(I)
    Gm = np.sqrt(Ix*Ix + Iy*Iy)
    Go = compute_grad_ori(Ix, Iy, Gm, b=8)
    return Gm, Go

In [ ]:
Gm, Go = compute_grad_mod_ori(I)
plt.imshow(Gm)
plt.colorbar()
plt.show()
plt.imshow(Go)
plt.colorbar()
plt.show()

In [ ]:
def compute_sift_region(Gm, Go, mask=None):
    # TODO
    sift = []
    # Note: to apply the mask only when given, do:
    if mask is not None:
        Gm = Gm*mask
    for i in range(16):
        subregion_gm = Gm[4*(i//4):4*(i//4)+4, 4*(i%4):4*(i%4)+4]
        subregion_go = Go[4*(i//4):4*(i//4)+4, 4*(i%4):4*(i%4)+4]
        histo = [0]*8
        for j in range(16):
            orientation = subregion_go[j//4, j%4]
            histo[orientation] += subregion_gm[j//4, j%4]
        sift.extend(histo)
    # éventuellement, utiliser linalg norm
    sift = np.array(sift)
    n = np.sqrt(np.sum(sift**2))
    if n < 0.5:
        sift = [0]*128
        return sift
    else:
        sift = sift/n
        sift[sift > 0.2] = 0.2
        sift = sift / np.sqrt(np.sum(sift**2))
    return sift

In [ ]:
# Example of viz of SIFTs
# set gausm to True to apply mask weighting of gradients
display_sift_region(I,           compute_grad_mod_ori, compute_sift_region, x=200, y=78, gausm=False)
display_sift_region(marche_im(), compute_grad_mod_ori, compute_sift_region, x=100, y=125, gausm=False)
display_sift_region(marche_im(), compute_grad_mod_ori, compute_sift_region, x=100, y=125, gausm=False)
display_sift_region(marche_im(), compute_grad_mod_ori, compute_sift_region, x=125, y=100, gausm=False)
display_sift_region(marche_im(), compute_grad_mod_ori, compute_sift_region, x=121, y=121, gausm=False)
display_sift_region(toy_im(),    compute_grad_mod_ori, compute_sift_region, x=95, y=95, gausm=False)

In [ ]:
def compute_sift_image(I):
    x, y = dense_sampling(I)
    im = auto_padding(I)
    
    # TODO calculs communs aux patchs
    Gm, Go = compute_grad_mod_ori(im)

    sifts = np.zeros([len(x), len(y), 128])
    for i, xi in enumerate(x):
        for j, yj in enumerate(y):
            #print(xi,yj)
            #print(Gm[xi:xi+16, yj:yj+16])
            sifts[i, j, :] = compute_sift_region(Gm[xi:xi+16, yj:yj+16], Go[xi:xi+16, yj:yj+16], mask=None)
    return sifts

In [ ]:
compute_sift_image(I)

# Partie 2 : Dictionnaire visuel

In [ ]:
# Ideally, restart the Python kernel to start from a clean slate
import os
import numpy as np
from tools import *
from sift import *
from sklearn.cluster import KMeans

In [ ]:
# Paths config
dir_sc = os.path.join('data', 'Scene')
dir_sift = os.path.join('data', 'sift')
path_vdict = os.path.join('data', 'kmeans', 'vdict.npy')
path_vdsift = os.path.join('data', 'kmeans', 'vdsift.npy')
path_vdinames = os.path.join('data', 'kmeans', 'vdinames.npy')

inames, ilabls, cnames = load_dataset(dir_sc)

In [ ]:
# If you want to look at the SIFTs that will be fed to compute_visual_dict:
sifts_list_by_image = compute_load_sift_dataset(dir_sc, dir_sift, inames, compute_sift_image)

In [ ]:
# Code here the `compute_visual_dict` function
def compute_visual_dict(sift, n_clusters=1000, n_init=1, verbose=1):
    # reorder data
    dim_sift = sift[0].shape[-1]
    sift = [s.reshape(-1, dim_sift) for s in sift]
    sift = np.concatenate(sift, axis=0)
    # remove zero vectors
    keep = ~np.all(sift==0, axis=1)
    sift = sift[keep]
    # randomly pick sift
    ids, _ = compute_split(sift.shape[0], pc=0.05)
    sift = sift[ids]
    #sift = sift/255
    
    kmeans = KMeans(n_clusters=n_clusters, n_init=n_init, verbose=verbose, n_jobs=-1).fit(sift)
    cc = kmeans.cluster_centers_
    n = 128
    return np.append(cc, np.zeros(n).reshape(1,n), axis=0)
    # TODO compute kmeans on `sift`, get cluster centers, add zeros vector

In [ ]:
# Run the visual dict computation (saved the first time)
vdict = compute_or_load_vdict(dir_sc, dir_sift, inames, compute_sift_image, path_vdict, compute_visual_dict)

In [ ]:
def closest_n_sift(sift, sifts, n):
    dist_2 = np.sum((sifts - sift)**2, axis=1)
    return np.argpartition(dist_2, n)[:n] #???

In [ ]:
# Study of the dict
regions, sifts = get_regions_and_sifts(dir_sc, np.random.choice(inames, 30)) # Compute SIFT and regions from 30 random images
#display_images(regions[np.random.choice(len(regions), 100)]) # Show 100 random regions

# TODO visually analyze the dict through similar example regions
#
# Ideally, create `vdregions`, a numpy array of size 1001 x 16 x 16 where each of the 1001 elements is a
# "prototype" patch that is very close to the cluster center
#vdregions = 

vdregions = np.empty((1001, 16, 16))

for i, c in enumerate(vdict):
    dist_2 = np.sum((sifts - c)**2, axis=1)
    vdregions[i] = regions[np.argmin(dist_2)]
    
display_images(vdregions)

# Partie 3 : BoW

In [ ]:
# Compute BoW for an image

def compute_feats(vdict, image_sifts):
    # TODO compute BoW from `image_sifts`
    # coding
    h = np.zeros((len(image_sifts), len(vdict)))
    for idx, sift in enumerate(image_sifts):
        # vérifier shape 1000*128 ?
        #hi = np.array([0]*len(vdict))
        h[idx][np.argmin(np.sum((vdict - sift)**2, axis=1))] = 1

    # pooling
    z = h.sum(axis=0)
    
    
    
    return z

In [ ]:
# Visualize your BoW on an image

iname = inames[0]
ipath = os.path.join(dir_sc, iname)
im = read_grayscale(ipath)
sift = compute_sift_image(im) * 255
regions = compute_regions(im)
feats = compute_feats(vdict, sift)

display_vdregions_image(im, vdict, sift, feats, vdregions=None) # if you have vdregions, add it as input here